## Import Library

In [1]:
# 将文件_lib.py里的函数导入到本文件中
from machine_lib import * 
import numpy as np

/Users/daijunpeng/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## 1, Login
填写用户名和密码与服务器建立连接session，相当于在网页上的登录操作

1, 去machine_lib.py login() 填写 username & pwd

2, 保存修改后machine_lib.py文件

3, 回到本文件重启内核

**------要重启内核！！！！！-------**

4, 重新运行第一格 from machine_lib import *

5, 所有machine_lib里的改动都要进行上述1-4 步骤生效

In [2]:
# 观察返回信息看是否连接成功
# 之后所有的api操作都需要使用一个已经登录了的session
s = login()

#用region_dict去找到对应region和univsere作为simulation的setting
region_dict = {"usa" : ("USA", "TOP3000"),"asi": ("ASI", "MINVOL1M"),"eur": ("EUR", "TOP1200"),
               "glb": ("GLB", "TOP3000"),"hkg": ("HKG","TOP800"),"twn": ("TWN", "TOP500"), "jpn": ("JPN", "TOP1600"),
               "kor": ("KOR", "TOP600"), "chn": ("CHN", "TOP2000U"), "amr" : ("AMR", "TOP600")}

b'{"user":{"id":"SD75828"},"token":{"expiry":14400.0},"permissions":["TUTORIAL","WORKDAY"]}'
username:1012169960@qq.com logged inpassword:Ace144169


## 2, get data fields

通过api获取数据的信息并转化成pandas dataframe便于使用

In [22]:
# 得到地区为USA，股票池为TOP3000的数据集，数据集id为analyst4
df = get_datafields(s, dataset_id = 'model51', region='USA', universe='TOP3000', delay=1)
print(df)

                                 id  \
0             beta_last_30_days_spy   
1            beta_last_360_days_spy   
2             beta_last_60_days_spy   
3             beta_last_90_days_spy   
4      correlation_last_30_days_spy   
5     correlation_last_360_days_spy   
6      correlation_last_60_days_spy   
7      correlation_last_90_days_spy   
8      systematic_risk_last_30_days   
9     systematic_risk_last_360_days   
10     systematic_risk_last_60_days   
11     systematic_risk_last_90_days   
12   unsystematic_risk_last_30_days   
13  unsystematic_risk_last_360_days   
14   unsystematic_risk_last_60_days   
15   unsystematic_risk_last_90_days   

                                          description  \
0                              Beta to SPY in 30 Days   
1                             Beta to SPY in 360 Days   
2                              Beta to SPY in 60 Days   
3                              Beta to SPY in 90 Days   
4                       Correlation to SPY in 30 Da

#### 数据预处理
1，data type matrix or vector\
2，ts_backfill 回填提高coverage\
3，winsorize 去极值

In [23]:
print(df[df['type'] == "MATRIX"]["id"].tolist())
# 回填处理数据集中type为matrix和vector的数据，并得到处理后的数据字段
pc_fields = process_datafields(df, "matrix")
pc_fields += process_datafields(df, "vector")
print(pc_fields)

['beta_last_30_days_spy', 'beta_last_360_days_spy', 'beta_last_60_days_spy', 'beta_last_90_days_spy', 'correlation_last_30_days_spy', 'correlation_last_360_days_spy', 'correlation_last_60_days_spy', 'correlation_last_90_days_spy', 'systematic_risk_last_30_days', 'systematic_risk_last_360_days', 'systematic_risk_last_60_days', 'systematic_risk_last_90_days', 'unsystematic_risk_last_30_days', 'unsystematic_risk_last_360_days', 'unsystematic_risk_last_60_days', 'unsystematic_risk_last_90_days']
['winsorize(ts_backfill(beta_last_30_days_spy, 120), std=4)', 'winsorize(ts_backfill(beta_last_360_days_spy, 120), std=4)', 'winsorize(ts_backfill(beta_last_60_days_spy, 120), std=4)', 'winsorize(ts_backfill(beta_last_90_days_spy, 120), std=4)', 'winsorize(ts_backfill(correlation_last_30_days_spy, 120), std=4)', 'winsorize(ts_backfill(correlation_last_360_days_spy, 120), std=4)', 'winsorize(ts_backfill(correlation_last_60_days_spy, 120), std=4)', 'winsorize(ts_backfill(correlation_last_90_days_spy,

## 3, Alpha factory
Factory是一个将数据字段，操作符，参数按照一定的方式组合成可以回测的alpha表达式的python方法。\
这里的first order（一阶）指的是直接把数据字段放进操作符中的最简单的组合方式，只抽取一维的信息。

In [24]:
# 将数据字段pc_fields使用操作符ts_ops和basic_ops(见machine_lib最前面)进行全组合，得到一阶的alpha表达式
first_order = first_order_factory(pc_fields, ts_ops+basic_ops)
# 随机打乱first_order的顺序，因为上边处理会使同一个数据字段扎堆出现，不利于短时间内回测更多种alpha
np.random.shuffle(first_order)
# 输出因子表达式列表前10个（只是用来看看长什么样）
print(first_order[:10])
print(len(first_order))

['ts_mean(winsorize(ts_backfill(beta_last_60_days_spy, 120), std=4), 22)', 'ts_mean(winsorize(ts_backfill(systematic_risk_last_90_days, 120), std=4), 66)', 'ts_product(winsorize(ts_backfill(unsystematic_risk_last_90_days, 120), std=4), 240)', 'ts_std_dev(winsorize(ts_backfill(systematic_risk_last_60_days, 120), std=4), 22)', 'ts_rank(winsorize(ts_backfill(beta_last_90_days_spy, 120), std=4), 66)', 'ts_arg_max(winsorize(ts_backfill(systematic_risk_last_60_days, 120), std=4), 120)', 'ts_std_dev(winsorize(ts_backfill(unsystematic_risk_last_30_days, 120), std=4), 120)', 'ts_arg_max(winsorize(ts_backfill(correlation_last_60_days_spy, 120), std=4), 22)', 'ts_rank(winsorize(ts_backfill(systematic_risk_last_60_days, 120), std=4), 5)', 'ts_scale(winsorize(ts_backfill(beta_last_90_days_spy, 120), std=4), 66)']
1008


##### 将factory生成的alpha表达式配以一个初始的decay值作为tuple元组，并以list的形式作为value放入以region为key的字典作为数据结构

In [25]:
# Load with alphas with regions and decay
# 对每个因子表达式增加初始的decay值设置
fo_alpha_dict = defaultdict(list)
init_decay = 3
regions = ["usa"]
for key in regions:
    for alpha in first_order:
        fo_alpha_dict[key].append((alpha,init_decay))

for key, value in fo_alpha_dict.items():
    print("%s : %d"%(key, len(value)))

usa : 1008


## 4, simulate alphas

请阅读machine_lib中的simulate方法理解代码逻辑

In [29]:
# 对上边生成的因子进行回测，并标记回测的name为analyst4_fo_usa（可以自己修改，只是个标记用以筛选）
# region_dict在上边定义过的，不用动
# 回测的中性化设置为SUBINDUSTRY（如果想要也可以修改，但没必要，成为顾问后会有新的代码给）
# 从第0个因子开始回测，将回测结果放进first_layer_bag中
first_layer_bag = []
fo_alpha_dict_unSimulated = dict(fo_alpha_dict)
simulate(fo_alpha_dict, region_dict, "model51_usa", 'SUBINDUSTRY', 0, first_layer_bag)

b'{"user":{"id":"SD75828"},"token":{"expiry":14400.0},"permissions":["TUTORIAL"]}'
username:1012169960@qq.com logged inpassword:Ace144169
curr usa len 1008
group 0 ts_mean(winsorize(ts_backfill(beta_last_60_days_spy, 120), std=4), 22) USA TOP3000 3
group 0 ts_mean(winsorize(ts_backfill(systematic_risk_last_90_days, 120), std=4), 66) USA TOP3000 3
group 0 ts_product(winsorize(ts_backfill(unsystematic_risk_last_90_days, 120), std=4), 240) USA TOP3000 3
group 0 post done
https://api.worldquantbrain.com/simulations/Z7XYBb9w4PqcFALQLdxXFU done simulating, getting alpha details
https://api.worldquantbrain.com/simulations/4FBTKjeZJ4Gi8UZfdF0mER9 done simulating, getting alpha details
https://api.worldquantbrain.com/simulations/3XkbATaAp4IZ8K21dtAYovhw done simulating, getting alpha details
group 0 USA simulate done
group 1 ts_std_dev(winsorize(ts_backfill(systematic_risk_last_60_days, 120), std=4), 22) USA TOP3000 3
group 1 ts_rank(winsorize(ts_backfill(beta_last_90_days_spy, 120), std=4), 66

ProxyError: HTTPSConnectionPool(host='api.worldquantbrain.com', port=443): Max retries exceeded with url: /simulations/1gk1Oacvt5888Dy1bxL5gGG5 (Caused by ProxyError('Unable to connect to proxy', RemoteDisconnected('Remote end closed connection without response')))

In [20]:
test_alpha = dict(fo_alpha_dict_unSimulated)
fo_alpha_dict_unSimulated.clear()
simulate_new(test_alpha, fo_alpha_dict_unSimulated, region_dict, "model51_usa", 'SUBINDUSTRY', 0)

b'{"user":{"id":"SD75828"},"token":{"expiry":14400.0},"permissions":["TUTORIAL"]}'
username:1012169960@qq.com logged inpassword:Ace144169
alpha_dict_unSimulated:
0


{}

## 5, Select alphas
go to web alphas penal to look for the number and date to track for next order improve

In [3]:
## get promising alphas to improve in the next order
# 选择时间区间内满足sharpe>0.5, fitness>0.4的已回测过的因子，获取的最大数量为500个
# 这里的时间区间内如果你没有回测或是没有满足条件的回测过的因子，则返回空
# 所以需要你先跑上边的一阶因子，之后才有东西进行二阶回测
fo_tracker = get_alphas("2025-04-01", "2025-04-02", 0.5, 0.4, "USA", 500, "track")

b'{"user":{"id":"SD75828"},"token":{"expiry":14400.0},"permissions":["TUTORIAL","WORKDAY"]}'
username:1012169960@qq.com logged inpassword:Ace144169
0
['RAYrRva', 'group_neutralize(-ts_rank(winsorize(ts_backfill(pv13_custretsig_retsig, 120), std=4), 120),densify(pv13_r2_min20_3000_sector))', 1.62, 0.4914, 0.79, 0.000478, '2025-04-01T23:51:08-04:00', 12]
['Ne6Am5X', "group_rank(ts_mean(winsorize(ts_backfill(rel_num_comp, 120), std=4), 22),densify(bucket(group_rank(cap,sector),range='0,1,0.1')))", 1.52, 0.0252, 1.18, 0.006001, '2025-04-01T23:41:22-04:00', 3]
['AQb3xjg', "group_zscore(ts_sum(winsorize(ts_backfill(pv13_revere_key_sector_total, 120), std=4), 240),densify(bucket(group_rank(cap,sector),range='0,1,0.1')))", 1.29, 0.0367, 1.22, 0.006143, '2025-04-01T23:57:44-04:00', 3]
['zrXA2Zd', 'ts_arg_max(winsorize(ts_backfill(pv13_revere_comproduct_company, 120), std=4), 5)', 1.25, 0.3004, 1.77, 0.004013, '2025-04-01T02:54:47-04:00', 3]
['kJXKvxl', 'group_zscore(-ts_zscore(winsorize(ts_back

In [ ]:
# 对获取的因子进行预处理
print(len(fo_tracker['next']))
print(len(fo_tracker['decay']))
fo_layer = transform(fo_tracker['next'] + fo_tracker['decay'], 'usa')

## 6, Next order improvement
second order: ts_ops(field, days) -> group_ops(ts_ops(field, days), group)

In [ ]:
# 对获取的因子进行二阶加工，得到更复杂的二阶因子
so_alpha_dict = defaultdict(list)
for key, couples in fo_layer.items():
    for expr, decay in couples:
        for alpha in get_group_second_order_factory([expr], group_ops, key):
            so_alpha_dict[key].append((alpha,decay))

for key, value in so_alpha_dict.items():
    print("%s : %d"%(key, len(value)))
np.random.shuffle(so_alpha_dict['usa'])
print(so_alpha_dict['usa'][:3])

In [ ]:
#so_bag = []
fo_alpha_dict_unSimulated = dict(so_alpha_dict)

In [ ]:
# 回测二阶因子，参数含义和上边一样
# simulate second order alphas
test_alpha = dict(fo_alpha_dict_unSimulated)
fo_alpha_dict_unSimulated.clear()
simulate_new(test_alpha, fo_alpha_dict_unSimulated, region_dict, "news18_usa", 'SUBINDUSTRY', 0)
#simulate(so_alpha_dict, region_dict, "news18_usa", 'SUBINDUSTRY', 0, so_bag)

## Higher Order for improvement - Third Order
group_ops(ts_ops(field, days), group) -> trade_when(entre_event, group_ops(ts_ops(field, days), group), exit_event)

In [ ]:
# 三阶处理方式同理
## get promising alphas from second order to improve in the third order
so_tracker = get_alphas("2025-03-22", "2025-03-23", 0.7, 0.5, "USA", 500, "track")

print(len(so_tracker['next']))
print(len(so_tracker['decay']))

so_layer = prune(so_tracker['next'] + so_tracker['decay'], 'usa', 'anl', 5)
th_alpha_dirt = defaultdict(list)
for region, couples in so_layer.items():
    for expr, decay in couples:
        for alpha in trade_when_factory("trade_when",expr,region):
            th_alpha_dirt[region].append((alpha,decay))
np.random.shuffle(th_alpha_dirt['usa'])

### Simulate Third Order

In [ ]:
fo_alpha_dict_unSimulated = dict(th_alpha_dirt)

In [ ]:
# simulate second order alphas
#th_bag = []
test_alpha = dict(fo_alpha_dict_unSimulated)
fo_alpha_dict_unSimulated.clear()
simulate_new(test_alpha, fo_alpha_dict_unSimulated, region_dict, "news18_usa", 'SUBINDUSTRY', 0)
#simulate(th_alpha_dirt, region_dict, "pv1_usa", 'SUBINDUSTRY', 0, th_bag)

## 7, Get submittable alphas
check_submission and view alphas

In [ ]:
# get submitable alphas to check submission
th_tracker = get_alphas("2025-03-22", "2025-03-23", 1.25, 1, "USA", 500, "submit")

In [ ]:
# 得到上边获得的因子的alpha id 存放在stone_bag种
stone_bag = []
for alpha in th_tracker['next'] + th_tracker['decay']:
    stone_bag.append(alpha[0])
print(len(stone_bag))
gold_bag = []

# 对stone_bag中的因子检查是否可以提交，可以提交的因子会存放在gold_bag中
# 过程种输出为catch的因子id为测试失败的因子，也就是不能确定可不可以提交
# 所以这边建议直接用check文件直接把stone bag的因子全部尝试进行提交
#check_submission(stone_bag, gold_bag, 0)

In [ ]:
# look date and metrics to locate alphas in the web
# 查看可提交的因子的详细信息
view_alphas(gold_bag)
print(len(stone_bag))

In [ ]:
submit_alpha(stone_bag, 0)

## 8, fine-tune submittable alphas
neutralization, performance comparison, turnover, margin